In [1]:
import pandas as pd
import numpy as np
from llama_index.core import Document, SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.settings import Settings
from llama_index.core.chat_engine import CondenseQuestionChatEngine
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.llms.llama_cpp import LlamaCPP
from llama_cpp import Llama
from sentence_transformers import SentenceTransformer
from llama_index.embeddings.huggingface import HuggingFaceEmbedding    
import faiss

c:\Users\abhij\miniconda3\envs\RAG\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("./pubmed_abstracts/pubmed_abstracts.csv")

In [3]:
df = df.dropna(subset=["abstract"])

In [4]:
def dataframe_to_documents(df):
    docs = []
    for _, row in df.iterrows():
        text = row["abstract"]
        metadata = {
            "title": row.get("title", ""),
            "year": row.get("year", ""),
        }
        docs.append(Document(text=text, metadata=metadata))
    return docs

In [5]:
docs = dataframe_to_documents(df)

In [6]:
model_directory = r"E:\RAG_Models"
model_name = "phi-2-orange.Q4_K_M.gguf"

model_path = model_directory+"\\"+model_name

In [7]:
llm = LlamaCPP(model_path = model_path, temperature = 0.2, max_new_tokens = 256, context_window = 2048, verbose=True)

llama_model_loader: loaded meta data with 21 key-value pairs and 325 tensors from E:\RAG_Models\phi-2-orange.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi2
llama_model_loader: - kv   1:                               general.name str              = Phi2
llama_model_loader: - kv   2:                        phi2.context_length u32              = 2048
llama_model_loader: - kv   3:                      phi2.embedding_length u32              = 2560
llama_model_loader: - kv   4:                   phi2.feed_forward_length u32              = 10240
llama_model_loader: - kv   5:                           phi2.block_count u32              = 32
llama_model_loader: - kv   6:                  phi2.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi2.attention.head_count_kv u32  

In [8]:
Settings.llm = llm

In [9]:
Settings.embed_model = HuggingFaceEmbedding(model_name = "pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")

In [10]:
text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=50)
Settings.node_parser = text_splitter

In [11]:
dimensions = len(Settings.embed_model.get_text_embedding("sample text"))

faiss_index = faiss.IndexFlatL2(dimensions)
faiss_db = FaissVectorStore(faiss_index = faiss_index)
storage_context = StorageContext.from_defaults(vector_store = faiss_db)

In [12]:
index = VectorStoreIndex.from_documents(docs, storage_context=storage_context)

In [13]:
memory = ChatMemoryBuffer.from_defaults(token_limit = 600)
query_engine = index.as_query_engine(similarity_top_k = 3)

In [14]:
chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine = query_engine,
    memory = memory
)

In [15]:
query = 'Is elevated dopamine consistently observed in patients with schizophrenia?'

In [16]:
response = chat_engine.chat(query)

llama_perf_context_print:        load time =   31924.24 ms
llama_perf_context_print: prompt eval time =   31848.82 ms /  1090 tokens (   29.22 ms per token,    34.22 tokens per second)
llama_perf_context_print:        eval time =   31380.78 ms /   255 runs   (  123.06 ms per token,     8.13 tokens per second)
llama_perf_context_print:       total time =   63753.19 ms /  1345 tokens


In [20]:
response

AgentChatResponse(response="\nBased on the provided studies, there is evidence of elevated dopamine in patients with schizophrenia, particularly in midbrain regions that project to parts of the striatum. However, the relationship between dopamine and schizophrenia is complex, and the findings are not consistent across all studies. Some studies have found preliminary evidence of an association between chronic social stress exposure and increased striatal dopamine functioning, but the quality of available studies is generally low. Therefore, while there is some evidence of elevated dopamine in schizophrenia, the findings are not universally consistent.\n\nThe studies also suggest that NM levels are higher in patients with schizophrenia than in healthy control individuals, which further supports the link between dopamine and schizophrenia. However, it's important to note that these findings are based on specific measures of dopamine and NM, and more research is needed to fully understand 

In [ ]:
chat_response = response.response

"\nBased on the provided studies, there is evidence of elevated dopamine in patients with schizophrenia, particularly in midbrain regions that project to parts of the striatum. However, the relationship between dopamine and schizophrenia is complex, and the findings are not consistent across all studies. Some studies have found preliminary evidence of an association between chronic social stress exposure and increased striatal dopamine functioning, but the quality of available studies is generally low. Therefore, while there is some evidence of elevated dopamine in schizophrenia, the findings are not universally consistent.\n\nThe studies also suggest that NM levels are higher in patients with schizophrenia than in healthy control individuals, which further supports the link between dopamine and schizophrenia. However, it's important to note that these findings are based on specific measures of dopamine and NM, and more research is needed to fully understand the relationship between do